In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

vehicles_data_set = pd.read_csv('/home/jovyan/work/datasets/vehicles_cleaned.csv')

vehicles_data_set.head()

,price,year,manufacturer,fuel,odometer,title_status,transmission,type,state
0,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [2]:
X = vehicles_data_set.copy()

y = vehicles_data_set['price']

X.drop(columns=['price'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.6,
    random_state=42
)

X_test, X_val, y_test, y_val = train_test_split(
    X_test,
    y_test,
    train_size=0.5,
    random_state=42
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((256128, 8), (256128,), (85376, 8), (85376,), (85376, 8), (85376,))

In [3]:
cols_with_null_values = [col for col in X_train.columns
                        if X_train[col].isnull().any()]

for col_with_null_values in cols_with_null_values:
    print(
        X_train[col_with_null_values].value_counts(dropna=False)
    )

year
2018.0    21799
2017.0    21747
2015.0    19016
2013.0    18486
2016.0    18367
          ...  
1920.0        1
1913.0        1
1918.0        1
1901.0        1
1943.0        1
Name: count, Length: 112, dtype: int64
manufacturer
ford               42587
chevrolet          33119
toyota             20426
honda              12784
jeep               11603
nissan             11405
ram                10902
NaN                10624
gmc                 9975
bmw                 8756
dodge               8255
mercedes-benz       7066
hyundai             6149
subaru              5702
volkswagen          5598
kia                 5097
lexus               4911
audi                4630
cadillac            4172
chrysler            3560
acura               3547
buick               3290
mazda               3281
infiniti            2911
lincoln             2509
volvo               2069
mitsubishi          1968
mini                1416
pontiac             1384
rover               1284
jaguar           

In [4]:
X_train.reset_index(inplace=True, drop=True)

,year,manufacturer,fuel,odometer,title_status,transmission,type,state
0,2014.0,kia,gas,115166.0,clean,automatic,sedan,ca
1,2004.0,infiniti,gas,202.0,rebuilt,automatic,sedan,fl
2,2019.0,toyota,gas,20065.0,clean,automatic,SUV,ca
3,2018.0,chevrolet,gas,93000.0,clean,automatic,sedan,mt
4,2012.0,ford,gas,98549.0,clean,automatic,truck,fl


In [7]:
rows_fuel_with_null_values = X_train[X_train['fuel'].isnull()].index.to_list()

X_train_fuel_most_frequent_imputed = X_train.copy()

print(
    X_train_fuel_most_frequent_imputed['fuel'].isnull().any()
)

simple_imputer_most_frequent = SimpleImputer(strategy='most_frequent', add_indicator=True)

simple_imputer_most_frequent_values = simple_imputer_most_frequent.fit_transform(
    X_train_fuel_most_frequent_imputed['fuel']
)

X_train_fuel_most_frequent_imputed.drop(columns=['fuel'], inplace=True)

X_train_fuel_most_frequent_imputed = pd.concat(
    [
        X_train_fuel_most_frequent_imputed,
        pd.Series(
            simple_imputer_most_frequent_values[:,0],
            name='fuel'
        ),
        pd.Series(
            simple_imputer_most_frequent_values[:,1],
            name='fuel_missing'
        )
    ], axis=1
)

print(
    X_train_fuel_most_frequent_imputed['fuel'].isnull().any()
)

for row_fuel_with_null_values in rows_fuel_with_null_values:
    print(
        X_train['fuel'][row_fuel_with_null_values],
        '-----------',
        X_train_fuel_most_frequent_imputed['fuel'][row_fuel_with_null_values],
        '-----------',
        X_train_fuel_most_frequent_imputed['fuel_missing'][row_fuel_with_null_values]
    )

True


ValueError: Expected 2D array, got 1D array instead:
array=['gas' 'gas' 'gas' ... 'gas' 'gas' 'diesel'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.